#TODO: Maybe try this - https://youtu.be/0VLAoVGf_74?si=zuJ8AL_wLbsbRdd5
Compare MHA, MQA, MLA (DeepSeek)

## Part 2: Apply TransformerClassifier (Encoder Only) 

In [ ]:
from common_utils import *
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from transformer import TransformerClassifierMHA
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import OneCycleLR

### Step 1: Prepare Dataset Loader

In [ ]:
crowd_dataset = load_dataset("csv", data_files="./dataset/text_emotion.csv")
crowd_dataset = crowd_dataset.rename_column('content', 'text')
crowd_dataset_dict = create_train_validation_test(crowd_dataset['train'])

embedding_matrix = np.load(EMBEDDING_PATH)

with open(WORD2IDX_PATH, "r", encoding="utf-8") as f:
    word2idx = json.load(f)

Train size: 28000
Validation size: 4000
Test size: 8000


In [ ]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(crowd_dataset_dict['train']['sentiment'])
val_labels = label_encoder.transform(crowd_dataset_dict['validation']['sentiment'])
test_labels = label_encoder.transform(crowd_dataset_dict['test']['sentiment'])
num_classes = len(label_encoder.classes_)
print(f"Number of sentiment classes: {num_classes}")
print(f"Emotion classes: {label_encoder.classes_}")

crowd_labels_dict = {
    'train': train_labels,
    'validation': val_labels,
    'test': test_labels
}

Number of sentiment classes: 13
Emotion classes: ['anger' 'boredom' 'empty' 'enthusiasm' 'fun' 'happiness' 'hate' 'love'
 'neutral' 'relief' 'sadness' 'surprise' 'worry']


In [ ]:
crowd_dataloaders_dict = create_dataloaders(crowd_dataset_dict, crowd_labels_dict, word2idx)

Created DataLoaders with 875 training batches, 125 validation batches, and 250 test batches.


### Step 2: Train the TransformerClassifier

In [ ]:
TransformerClassifier_MHA = TransformerClassifierMHA(len(word2idx), 13, 100, 5, 3, 200, 100, 0.1)  
# vocab_size, num_classes, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

TransformerClassifier_MHA.to(device)

embedding_matrix_tensor = torch.FloatTensor(embedding_matrix)
TransformerClassifier_MHA.embedding.weight.data.copy_(embedding_matrix_tensor)
TransformerClassifier_MHA.embedding.weight.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(TransformerClassifier_MHA.parameters(), lr=0.001)

total_steps = 100 * len(crowd_dataloaders_dict['train'])
scheduler = OneCycleLR(
    optimizer,
    max_lr=0.001,
    total_steps=total_steps,
    pct_start=0.1,      
    anneal_strategy='cos'
)

Using device: cpu


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=100):
    since = time.time()
    best_model_wts = model.state_dict()
    best_val_f1 = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()  
                
            running_loss = 0.0
            all_preds = []
            all_labels = []
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                
                running_loss += loss.item() * inputs.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(all_labels, all_preds)
            epoch_f1 = f1_score(all_labels, all_preds, average='weighted')
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {epoch_f1:.4f}')
            
            if phase == 'validation':
                if epoch_f1 > best_val_f1:
                    best_val_f1 = epoch_f1
                    best_model_wts = model.state_dict().copy()
                    
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'val_f1': epoch_f1,
                    }, 'best_transformer_MHA_model.pt')
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val F1: {best_val_f1:.4f}')

    model.load_state_dict(best_model_wts)
    return model

trained_model = train_model(TransformerClassifier_MHA, crowd_dataloaders_dict, criterion, optimizer, scheduler, num_epochs=100)

def evaluate_model(model, test_dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_acc = accuracy_score(all_labels, all_preds)
    test_f1 = f1_score(all_labels, all_preds, average='weighted')
    
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    
    return test_acc, test_f1

test_acc, test_f1 = evaluate_model(trained_model, crowd_dataloaders_dict['test'])